# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,paamiut,61.9940,-49.6678,33.37,75,100,4.25,GL,1680574652
1,1,albany,42.6001,-73.9662,54.01,62,100,4.99,US,1680574462
2,2,busselton,-33.6500,115.3333,68.31,53,38,9.84,AU,1680574652
3,3,lobva,59.1854,60.5185,35.46,96,100,8.03,RU,1680574652
4,4,changying,52.8733,123.3213,26.06,49,2,14.47,CN,1680574653


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lat", "Long", size="Humidity", tiles="OSM", geo=True, color="city", colormap="rainbow")

# Display the map
humidity_map

ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 75)]
ideal_data_df = city_data_df.loc[city_data_df["Wind Speed"] < 5]

# Drop any rows with null values
ideal_data_df = ideal_data_df.dropna()

# Display sample data
ideal_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,cap malheureux,-19.9842,57.6142,75.47,82,89,1.99,MU,1680574198
31,31,srandakan,-7.9386,110.2506,80.94,74,100,2.89,ID,1680574662
41,41,tessalit,20.1986,1.0114,75.67,7,0,4.14,ML,1680574663
42,42,hilo,19.7297,-155.0900,78.94,71,100,3.44,US,1680574118
44,44,dalby,-27.1833,151.2667,83.98,38,0,1.01,AU,1680574663


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_data_df.copy()
hotel_df = hotel_df.drop(columns=["City_ID","Cloudiness", "Date","Max Temp", "Wind Speed"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
15,cap malheureux,-19.9842,57.6142,82,MU,
31,srandakan,-7.9386,110.2506,74,ID,
41,tessalit,20.1986,1.0114,7,ML,
42,hilo,19.7297,-155.0900,71,US,
44,dalby,-27.1833,151.2667,38,AU,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cap malheureux - nearest hotel: No hotel found
srandakan - nearest hotel: No hotel found
tessalit - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
dalby - nearest hotel: No hotel found
ghatsila - nearest hotel: No hotel found
bhatkal - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
thinadhoo - nearest hotel: No hotel found
meulaboh - nearest hotel: No hotel found
bima - nearest hotel: No hotel found
mutata - nearest hotel: No hotel found
padang - nearest hotel: No hotel found
villamontes - nearest hotel: No hotel found
grand gaube - nearest hotel: No hotel found
victoria - nearest hotel: No hotel found
beaufort - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
gamba - nearest hotel: No hotel found
sao desiderio - nearest hotel: No hotel found
nichlaul - nearest hotel: No hotel found
voh - nearest hotel: No hotel found
manjeshwar - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel

,City,Lat,Lng,Humidity,Country,Hotel Name
15,cap malheureux,-19.9842,57.6142,82,MU,No hotel found
31,srandakan,-7.9386,110.2506,74,ID,No hotel found
41,tessalit,20.1986,1.0114,7,ML,No hotel found
42,hilo,19.7297,-155.0900,71,US,No hotel found
44,dalby,-27.1833,151.2667,38,AU,No hotel found
...,...,...,...,...,...,...
557,cukai,4.2500,103.4167,72,MY,No hotel found
561,soe,-9.8607,124.2840,65,ID,No hotel found
563,narrabri,-30.3167,149.7833,53,AU,No hotel found
567,sabang,5.8933,95.3214,74,ID,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Long", "Lat", hover_cols=["Country", "Hotel Name"], color="City", size="Humidity", geo=True, tiles = "OSM")

# Display the map
hotel_map

ImportError: Geographic projection support requires GeoViews and cartopy.